# [INFO-H515 - Big Data Scalable Analytics](https://uv.ulb.ac.be/course/view.php?id=85246?username=guest)

## TP 3 - Streaming forecasting (RLS and ML) with Kafka and Spark Streaming

#### *Gianluca Bontempi, Jacopo De Stefani and Theo Verhelst*

####  29/04/2020

## Sending data to Kafka

This notebook uses the [Python client for the Apache Kafka distributed stream processing system](http://kafka-python.readthedocs.io/en/master/index.html) to send messages to a Kafka cluster. 

In this example, the messages are data generated from a linear model with $n$ input variables, i.e., 

$$
y =x^T \beta +w
$$
with $x, \beta \in \mathbb{R}^n$, and $y, w \in \mathbb{R}$. $w$ is Gaussian noise.

Messages are sent every `time_delay` seconds. They are a list of size $(n+2)$ where:
* First element is the message index 
* Second element is $y$ 
* Third to last elements are $x$ values (size $n$)


Let's start by importing all the required libraries

In [1]:
from kafka import KafkaProducer
import time
import numpy as np

Then, let's create a Kafka Producer, running on port 9092, in order to be able to send messages.

In [2]:
producer = KafkaProducer(bootstrap_servers='localhost:9092')

## Linear DGP (Data Generating Process)

In this example, the messages are data generated from a linear model with $n$ input variables and random coefficients $\beta$, i.e., 

$$
y =x^T \beta +w
$$
with $x, \beta \in \mathbb{R}^n$, and $y, w \in \mathbb{R}$. $w$ is Gaussian noise.

Please note that the numerical values, here encoded as a numpy array, are sent to Kafka in a serialized (string) format.


In [5]:
np.random.seed(2452020515) # Fix seed to ensure repeatability
i=0 #Initialise counter

n=10   # number of inputs
time_delay = 3 # Time delay between the transmission of two consecutive messages

beta=np.zeros(n) 
beta[0]=1   
beta[-1]=1 ## first and last parameters are 1, others are zeros
beta.shape=(n,1)


#Infinite loop for sending messages to Kafka with the topic dataLinearModel
while True:
    # Randomly generate x_i
    x=np.random.randn(1,n)[0]
    
    # Compute y from x_i according to formula
    y=float(x.dot(beta))+0.1*np.random.rand(1)[0] ## y =x^T beta +w

    # Serialize array and print message as a string
    message=np.array2string(np.append([i,y],x),separator=",") 
    print(message) # n=10 -> 12 elements in the message: cnt+y+10 xi
    
    # Send message on the dataLinearModel topic
    producer.send('dataLinearModel', message.encode())
    
    i=i+1
    time.sleep(time_delay)
    

[ 0.        ,-0.69070552, 0.14116957,-0.35972144, 0.23525346,-0.84060208,
  1.31307295, 0.21407109,-0.83490104, 0.32611369, 0.55512634,-0.87705984]
[ 1.        ,-1.19783834, 0.60197096, 1.23855365,-1.02675744,-0.13221217,
  0.05266952, 0.31801924,-0.15956333, 0.48534895,-0.98568595,-1.82303797]
[ 2.        , 1.40743024, 0.89640743, 0.59349958, 1.23150731,-0.48190073,
 -0.83242817, 0.5046102 , 0.26038583, 0.20206037, 0.20707494, 0.43969144]
[ 3.        , 1.57545697, 0.48283181, 0.96588944,-2.71362256,-0.5599426 ,
  0.88906694,-0.98401467,-1.06144788, 0.30031018,-0.72943079, 1.00139819]
[ 4.        ,-0.15583584, 1.17130617,-0.63411887, 0.25766715,-0.15405764,
 -0.51831706,-1.12815054,-0.86595985,-0.17748257,-0.77464225,-1.34240026]
[ 5.        ,-0.12125809,-0.42194064, 1.07354404,-0.82047922,-1.45382133,
 -1.72097765,-0.41379372,-0.34063067,-0.95055884,-1.03530299, 0.29369309]
[ 6.        ,-0.06661557,-0.8922547 ,-1.43970548,-1.11521615,-0.71218819,
 -1.36137446,-0.07303445, 1.1461213 , 

[ 5.20000000e+01, 2.94232523e-01, 3.75833876e-01, 5.71650378e-01,
 -5.11931179e-02, 1.85460075e+00, 1.67596334e-01,-9.90930924e-02,
 -6.02053593e-01, 4.79714793e-02,-5.62930749e-01,-1.69468197e-01]
[ 5.30000000e+01,-3.00884971e-01, 1.73264700e+00, 9.68939136e-01,
  6.53252369e-01, 2.19849567e-01, 1.45849120e+00, 3.53757661e-01,
 -6.06083353e-01, 2.65032410e-02, 1.06375441e+00,-2.11783344e+00]
[54.        , 1.1990315 , 1.07692231,-0.63298952,-1.20265019, 1.02630228,
 -0.67640214,-1.33660293,-2.13310559, 0.89289807, 0.43030715, 0.0954479 ]
[ 5.50000000e+01,-8.11494112e-01,-6.58434579e-01, 3.27324678e-01,
  4.55258144e-01,-1.63783758e-01, 9.34701217e-01,-5.46775800e-01,
  3.37085002e-01, 2.04832910e-02,-7.70611709e-01,-2.28202635e-01]
[56.        , 1.45039478,-0.37123836, 0.54874216,-0.57703279, 0.87500119,
 -1.22243342,-0.95647864,-1.36587687, 0.35943697,-0.77315104, 1.80828586]
[57.        , 1.12674811,-0.23168403, 0.72280204,-1.00323228, 0.45073231,
  1.65794483,-0.58771456,-0.68306273

[ 1.00000000e+02, 1.85969780e-01,-1.29216093e-01,-4.46410384e-01,
 -9.41683270e-01, 5.57729401e-01,-1.14086580e+00,-1.93405082e+00,
 -6.99807599e-02,-1.53263894e+00, 1.46703741e+00, 2.76215458e-01]
[ 1.01000000e+02,-1.59427888e+00,-3.17083986e-01, 6.25046435e-01,
 -3.79748291e-02,-8.75042439e-01,-1.10801973e+00,-7.60882752e-02,
 -1.60962107e-01, 7.27362070e-01,-1.35513712e+00,-1.35259764e+00]
[102.        ,  2.81777132,  1.37436685, -0.61858997,  1.42019098,
   0.44077956, -2.39606032,  0.49056723,  0.74405796, -0.7909456 ,
  -0.18369568,  1.42723478]
[103.        ,  0.24991673, -0.78693837,  1.70682376, -0.56959001,
   1.16253372, -0.40603981,  0.23674303, -1.0785333 , -1.63430832,
  -0.77014208,  0.97539864]
[ 1.04000000e+02, 8.16612336e-01, 9.58938909e-02, 2.93132838e-01,
  3.22193701e-01, 1.84485875e+00, 4.21803465e-01, 1.13627836e+00,
 -9.34454123e-01, 8.83622021e-01, 9.40509897e-01, 7.03903419e-01]
[105.        ,  0.32622734, -0.26027519,  1.32932614,  0.86147575,
   0.57860376, 

[ 1.44000000e+02,-9.24436807e-01,-2.40351769e+00,-3.80756512e-01,
 -5.28075042e-01,-2.95336716e-01,-2.57509037e+00,-1.47898460e+00,
 -1.66485422e+00, 6.68672825e-04, 5.00307749e-01, 1.44994466e+00]
[ 1.45000000e+02, 5.69972589e-01, 4.00857516e-01,-8.18873034e-01,
 -1.34319049e+00,-1.27356376e-01, 1.28224792e+00, 1.94861010e-01,
 -1.41424530e-01, 1.01606265e+00, 6.01036062e-01, 1.46245723e-01]
[ 1.46000000e+02,-9.37672927e-01, 1.78707772e-01,-9.77698789e-01,
  1.25042514e-01,-1.01039174e+00,-1.00074201e+00, 4.21945365e-01,
  6.19300150e-01, 9.37780286e-01,-3.76095833e-01,-1.14598099e+00]
[ 1.47000000e+02,-3.76372529e+00,-2.70808193e+00, 8.58648798e-01,
 -2.13391905e-01,-4.94569517e-01, 1.41408444e+00, 1.72974131e+00,
 -1.13501914e+00, 1.34189848e+00,-5.19191294e-02,-1.12341614e+00]
[148.        ,  2.27059275,  0.54046934, -0.16586949,  0.25449666,
  -0.91458324,  0.82249203, -0.27567645,  0.88274716,  1.19675475,
   0.30147471,  1.70679567]
[ 1.49000000e+02,-2.69891040e+00,-1.09709416e+

[ 1.87000000e+02, 3.55912207e-02, 7.25681556e-01,-7.99615952e-01,
 -6.21690194e-01,-1.42109892e-01, 1.93773395e+00,-2.22823898e+00,
 -3.55073065e-01,-6.38752575e-01, 4.14933086e-01,-7.58365573e-01]
[ 1.88000000e+02,-3.66236120e-01,-1.52909073e-02,-6.80655891e-01,
  2.84674874e-01,-1.26502886e+00,-7.07489038e-01,-4.04004768e-01,
  1.30351461e+00, 1.47440578e+00, 1.36254206e+00,-4.16680162e-01]
[ 1.89000000e+02,-1.65044986e+00,-6.96140172e-03,-1.50690274e+00,
 -6.42024027e-01,-3.28194490e-01,-3.09494135e-01,-1.68477825e+00,
  5.78972379e-01,-7.95642714e-01, 9.84919058e-01,-1.67286776e+00]
[ 1.90000000e+02, 6.21177035e-01, 6.90849122e-01,-6.77464624e-01,
 -3.49220649e-01, 2.05805858e+00,-4.00995401e-01,-1.10163353e+00,
 -6.54950180e-01, 3.62565918e-01, 1.55941631e+00,-7.90980258e-02]
[ 1.91000000e+02,-2.63101754e+00,-7.87884824e-01,-5.07359939e-01,
  1.04510877e+00,-1.18989267e+00, 1.75285161e-01, 2.39096502e-01,
  1.02136149e+00, 2.91013988e-01,-2.18430814e-01,-1.87152043e+00]
[ 1.920000

...

[ 8.18000000e+02, 3.47940155e-01, 6.43378396e-01, 5.78746465e-01,
 -5.78307669e-01,-2.77299883e-01, 1.22203353e+00,-6.10262766e-03,
 -1.54016812e-01,-1.06969238e-01, 3.70617288e-01,-3.53669507e-01]
[ 8.19000000e+02, 3.46964606e+00, 2.64298604e+00, 1.89546053e+00,
 -1.50673404e+00,-2.59578254e-01,-1.16714853e+00,-1.45856107e+00,
  3.15068052e-01,-4.15444746e-01,-3.77225442e-02, 7.72113092e-01]
[ 8.20000000e+02, 2.16125529e+00, 1.42082356e+00, 1.06333000e+00,
  7.06392515e-01, 1.11295872e+00, 1.34402687e+00, 7.15249314e-01,
 -1.79874347e+00, 1.33307274e-01,-1.28259478e-01, 6.60765445e-01]
[ 8.21000000e+02, 3.73317607e-01,-2.44097351e-01, 2.83483861e-01,
 -7.71266089e-01,-1.84200501e+00, 1.29902776e+00,-1.56203583e-01,
 -2.00888014e-01, 1.65874819e+00,-1.26847763e-01, 5.92986352e-01]
[ 8.22000000e+02,-7.24828435e-01, 2.32223616e-01, 1.24407322e+00,
  1.93818618e-01,-5.96467128e-01, 4.03909982e-02, 4.47051873e-01,
  2.07880828e-01, 8.56206167e-01, 8.68607866e-02,-1.03532249e+00]
[ 8.230000

[ 8.60000000e+02,-2.31115346e+00,-1.73759830e+00,-1.42432866e-01,
 -1.13450897e+00, 9.44184776e-01, 4.68385055e-01, 1.43781046e+00,
  7.09723492e-01,-1.12382621e+00,-9.21029673e-01,-6.65237883e-01]
[ 8.61000000e+02,-7.02029868e-01,-1.23256809e+00, 4.47876540e-01,
  3.36704560e-01, 1.45916719e+00,-5.45974631e-01,-6.55680769e-01,
 -3.79478864e-02,-4.86262512e-01,-3.19245646e-01, 4.87452990e-01]
[ 8.62000000e+02,-1.22100976e+00,-6.74115556e-01,-3.08424583e-01,
  1.87799635e+00, 2.26713232e+00, 3.38232518e-01, 1.00994857e+00,
 -1.17273387e+00,-8.15811442e-01, 7.28703380e-01,-6.10545603e-01]
[ 8.63000000e+02,-1.11415355e+00,-8.13842464e-01,-7.83953300e-01,
 -1.52838370e-01,-8.45113062e-01,-2.09271992e+00, 1.27000010e+00,
 -1.43626109e+00, 1.65385345e+00,-7.52301150e-01,-3.12832802e-01]
[ 8.64000000e+02,-1.94039153e+00,-4.26807133e-01,-1.80839760e+00,
  7.97878561e-01,-8.36907555e-01,-8.87999127e-01,-1.48815817e+00,
  1.62263620e-02, 2.14105259e+00,-7.44661560e-01,-1.58985092e+00]
[ 8.650000

[ 9.02000000e+02,-1.43274417e+00,-1.88821546e+00, 1.63208801e+00,
 -3.17854445e-02, 6.88371792e-01, 2.27040797e-01, 7.06837828e-01,
  7.80653712e-01, 6.11953072e-01,-1.21572796e+00, 3.60844067e-01]
[ 9.03000000e+02, 1.38636040e-01, 2.20576858e-01,-1.99858421e-01,
 -2.66841419e-01,-1.43537232e+00,-7.50135912e-01,-2.06429416e-01,
 -2.67670861e+00, 2.72250982e-01, 1.47723012e+00,-1.80967776e-01]
[ 9.04000000e+02,-2.58634346e-01,-3.09058883e-01,-6.70769854e-01,
  1.55704277e-01, 1.36343825e-01,-3.41220138e-01, 5.08930645e-01,
 -5.85374707e-01, 4.38713042e-01, 2.30065010e-01,-4.32935196e-02]
[ 9.05000000e+02,-6.75773518e-01, 8.81713923e-01,-1.56798166e-01,
  1.42397248e-01, 5.62124148e-01,-3.57884360e-01,-1.06783180e+00,
 -9.83132859e-01,-1.28117829e+00,-1.06300516e+00,-1.59988877e+00]
[ 9.06000000e+02,-9.48755295e-01,-1.14634716e+00,-3.73768352e-01,
  1.10833099e+00, 9.62193149e-02, 1.50410015e+00, 5.98249907e-01,
 -1.48085066e+00, 8.48864918e-01,-1.08574411e+00, 1.22141643e-01]
[ 9.070000

[ 9.44000000e+02, 1.79586676e+00, 1.26736738e+00,-4.01045705e-01,
  1.07617447e+00,-7.70537930e-01, 6.98287995e-01,-7.32686147e-02,
  6.26857333e-01, 1.05014403e+00,-1.39219622e+00, 4.35197760e-01]
[ 9.45000000e+02, 5.52696984e-01, 7.25081943e-01, 1.29563749e+00,
 -1.74405907e+00,-1.62173094e+00,-1.82791535e+00,-1.46256743e+00,
  7.74892292e-01, 4.05536296e-02,-1.47768418e+00,-2.09070175e-01]
[ 9.46000000e+02,-3.54535263e-01, 4.28790294e-02, 9.41393782e-01,
 -3.66671610e-01,-1.33255672e+00, 9.18556103e-02, 3.05488658e-02,
 -2.66602096e+00, 4.76465804e-01,-7.44159326e-01,-4.50016560e-01]
[ 9.47000000e+02, 3.99495951e-01, 5.20794175e-02,-2.22797974e-02,
 -5.00336152e-01,-1.22847524e-01,-3.07126939e-01, 4.62130693e-01,
 -2.14036071e+00,-1.70383428e-01,-1.66052208e+00, 2.67886805e-01]
[ 9.48000000e+02, 4.68823474e+00, 3.00520291e+00,-1.72554375e+00,
  7.24738209e-01, 1.37148915e+00, 4.56112486e-01,-1.37718856e+00,
 -9.02836755e-01, 3.09387658e-02,-6.67875291e-02, 1.64373650e+00]
[ 9.490000

KeyboardInterrupt: 

**N.B** As the cell runs an infinite loop, the producer is never going to stop by itself. 
Don't forget to stop the cell using the dedicated button (■).

## Non-linear DGP (Data Generating Process)

In this example, the messages are data generated from a non-linear model with $n$ input variables, i.e., 

$$
y = \sin(x_0) + |x_1*x_2| + \sum_{i=2}^{10} log(x_i) + w
$$
with $x \in \mathbb{R}^n$, and $y, w \in \mathbb{R}$. $w$ is Gaussian noise.

Please note that the numerical values, here encoded as a numpy array, are sent to Kafka in a serialized (string) format.

In [6]:
np.random.seed(2452020515) # Fix seed to ensure repeatability
i=0 #Initialise counter

n=10   # number of inputs
time_delay = 3 # Time delay between the transmission of two consecutive messages

#Infinite loop for sending messages to Kafka with the topic dataNonLinearModel
while True:
    # Randomly generate x_i
    x=np.random.rand(1,n)[0]
    # Compute y from x_i according to formula
    y=float(np.sin(x[0])+abs(x[1]*x[2])+np.log(abs(x[-1])))+0.25*np.random.rand(1)[0]
    
    # Serialize array and print message as a string
    message=np.array2string(np.append([i,y],x),separator=",") 
    print(message) # n=10 -> 12 elements in the message: cnt+y+10 xi
    
    # Send message on the dataLinearModel topic
    producer.send('dataNonLinearModel', message.encode())
    
    i=i+1
    time.sleep(time_delay)
    

[ 0.        ,-2.13517467, 0.00241515, 0.02793932, 0.05161399, 0.67596521,
  0.10201457, 0.61138142, 0.55168554, 0.81702968, 0.09720055, 0.10013508]
[ 1.        ,-0.0655868 , 0.11901551, 0.17727414, 0.70426614, 0.45184756,
  0.77991384, 0.63604579, 0.45115242, 0.12065051, 0.98063141, 0.57960092]
[2.        ,0.3517806 ,0.35370735,0.34970193,0.41873636,0.23228666,
 0.20912698,0.96581699,0.87180643,0.02677749,0.70675843,0.81228294]
[3.        ,1.24464592,0.99681882,0.3823275 ,0.80071452,0.70452194,
 0.16261121,0.79832087,0.88443229,0.86393125,0.88721667,0.92640682]
[ 4.        ,-0.19919233, 0.7133137 , 0.83414193, 0.66703191, 0.48517621,
  0.42816018, 0.26098702, 0.71595058, 0.12970913, 0.91358122, 0.19263513]
[5.        ,0.65050537,0.60041567,0.1450803 ,0.77535642,0.29942698,
 0.9122697 ,0.34722958,0.78218832,0.24913355,0.91958351,0.9206573 ]
[6.        ,0.24952881,0.19094915,0.3580098 ,0.41742382,0.0971004 ,
 0.01398776,0.22867529,0.26245276,0.36292156,0.15258247,0.83366979]
[ 7.        

[ 5.40000000e+01,-3.33828053e+00, 4.01090687e-01, 9.02621164e-01,
  3.65317091e-01, 7.27098291e-02, 9.33678972e-01, 7.35904165e-01,
  1.12238687e-01, 1.23840433e-01, 9.67036742e-01, 1.54233976e-02]
[5.50000000e+01,5.80947147e-01,5.58925834e-01,6.92756406e-01,
 9.31522053e-01,2.66029023e-01,5.81767221e-01,8.74612004e-01,
 3.59192400e-01,1.93889424e-02,2.57472592e-01,5.21972139e-01]
[5.60000000e+01,2.90823969e-01,1.44287057e-02,4.75919419e-01,
 7.70949750e-01,9.26657042e-01,9.99011312e-01,4.02239453e-01,
 7.86516113e-01,5.07248470e-01,7.17969595e-01,7.75219885e-01]
[57.        ,-0.2871475 , 0.56836838, 0.78948733, 0.24200511, 0.47993862,
  0.89093742, 0.265453  , 0.37517218, 0.1453772 , 0.81263106, 0.34662138]
[58.        , 0.82409512, 0.33397159, 0.90046448, 0.58332387, 0.95650442,
  0.45190973, 0.78824625, 0.45101346, 0.34505792, 0.84087354, 0.89078818]
[59.        , 0.94762075, 0.97044376, 0.79673753, 0.69453109, 0.60308813,
  0.62578477, 0.13381012, 0.47040585, 0.54611166, 0.65747868

[1.03000000e+02,6.34138643e-02,9.31299325e-02,9.13665939e-01,
 9.37516417e-01,5.53578422e-01,7.10108313e-01,5.82878264e-01,
 5.36635574e-01,1.03653600e-01,5.63009780e-01,3.77594496e-01]
[ 1.04000000e+02,-2.06871689e+00, 4.95223029e-01, 6.39861619e-01,
  3.85340915e-01, 8.16240379e-01, 1.07710386e-01, 7.40959078e-01,
  4.10986103e-01, 5.96158808e-02, 4.10351675e-01, 5.69016683e-02]
[ 1.05000000e+02,-6.66634546e-01, 6.10506032e-01, 8.73129042e-01,
  5.46028170e-01, 8.24453820e-01, 4.01877223e-01, 9.99389830e-01,
  6.79945030e-02, 1.04324131e-01, 6.73312462e-01, 1.72595833e-01]
[106.        , -0.70216798,  0.14417713,  0.56792671,  0.78806854,
   0.77316477,  0.15738388,  0.61115483,  0.67465784,  0.93142442,
   0.81588742,  0.26751746]
[1.07000000e+02,7.93559073e-02,3.81039813e-01,4.17225509e-01,
 7.87107154e-01,8.36361023e-01,4.00664689e-01,1.61896370e-01,
 1.61916579e-02,7.07384183e-02,6.61254831e-01,4.67971590e-01]
[1.08000000e+02,3.72839414e-01,3.08627131e-01,2.30268505e-01,
 3.85474

[1.48000000e+02,9.72799375e-01,7.58229187e-01,4.27380428e-01,
 9.00503784e-01,4.15533686e-01,2.03832860e-01,1.23003066e-01,
 5.26430050e-02,5.23270323e-01,8.73315082e-01,8.52072081e-01]
[1.49000000e+02,1.04529937e+00,7.40280285e-01,6.00552526e-01,
 8.21511908e-01,7.91323371e-01,1.67290883e-01,6.37580308e-02,
 8.63616888e-01,4.09467636e-01,8.35632535e-01,7.20649216e-01]
[ 1.50000000e+02,-5.20141305e-01, 3.50031358e-01, 7.71043787e-01,
  9.93285201e-01, 3.75558732e-01, 6.45745948e-01, 7.67147688e-01,
  4.95495212e-01, 2.16480352e-01, 1.47981042e-01, 1.56722806e-01]
[ 1.51000000e+02,-3.31487329e-01, 6.63024072e-02, 9.74528974e-01,
  8.08201275e-02, 4.29712459e-02, 9.74596966e-02, 9.57710267e-01,
  7.46430468e-01, 2.70628683e-01, 7.02387767e-01, 5.07280640e-01]
[1.52000000e+02,1.00939556e+00,9.12642192e-01,4.98079770e-01,
 7.86426196e-01,7.75009908e-01,1.28256137e-01,4.07589639e-01,
 5.38313022e-01,7.45922993e-01,6.67300468e-01,7.36400296e-01]
[153.        ,  0.24093128,  0.56290455,  0.79

[ 1.93000000e+02,-1.90908287e+00, 8.28023918e-02, 1.76984898e-01,
  1.50604190e-01, 6.30443782e-01, 4.36576323e-01, 3.57101803e-01,
  8.90569535e-01, 8.51057355e-01, 7.93458520e-01, 1.30735333e-01]
[ 1.94000000e+02,-1.36132241e+00, 6.59493767e-01, 1.56909999e-01,
  2.93144754e-01, 7.35290115e-01, 7.12170007e-01, 2.34962367e-01,
  2.83302430e-01, 3.21463966e-01, 9.72809671e-01, 1.21622100e-01]
[1.95000000e+02,1.09022422e+00,8.02030336e-01,4.63232150e-01,
 6.41389513e-01,5.18586782e-01,5.75970067e-01,9.64963503e-01,
 7.70199996e-01,1.54417269e-01,7.07860755e-01,8.45683063e-01]
[ 1.96000000e+02,-2.30495276e-01, 1.80373564e-01, 1.68231823e-01,
  7.30365234e-02, 8.24547482e-01, 2.92740507e-01, 2.93922648e-01,
  9.13642051e-01, 8.79677270e-01, 4.86167893e-01, 5.82594305e-01]
[ 1.97000000e+02,-2.87934660e-01, 3.15100208e-02, 3.08938033e-01,
  5.35776131e-01, 9.16354281e-01, 1.85150413e-01, 3.33222813e-01,
  6.43377434e-01, 3.30755869e-01, 5.41910513e-02, 5.97017111e-01]
[198.        ,  0.5533

[ 2.36000000e+02,-5.99254161e-01, 4.11978353e-01, 7.87830349e-01,
  6.57349495e-01, 2.16589509e-01, 4.98690735e-01, 3.00156260e-01,
  1.09059608e-01, 2.63854636e-01, 4.56620045e-01, 1.82630036e-01]
[2.37000000e+02,5.10946749e-01,3.32050386e-01,5.98882158e-01,
 2.93793068e-01,2.22977408e-01,7.82495657e-01,9.07238195e-02,
 1.96535041e-01,6.65840254e-01,4.71859479e-01,9.91440741e-01]
[ 2.38000000e+02,-6.02077094e-01, 1.66781842e-01, 3.78708350e-01,
  7.10502096e-01, 1.19742566e-01, 4.86230783e-01, 7.71459896e-01,
  9.42593885e-02, 2.82655611e-01, 1.62483511e-01, 2.99325367e-01]
[ 2.39000000e+02,-2.68172153e+00, 8.94480678e-01, 7.89199586e-01,
  5.95612934e-02, 7.68099060e-01, 6.03347004e-01, 8.62713319e-01,
  2.95561352e-01, 4.76139347e-01, 2.83877159e-01, 2.47460995e-02]
[ 2.40000000e+02,-2.65935940e-01, 1.40617046e-01, 5.31744120e-03,
  2.63021282e-01, 7.33925283e-01, 1.06201524e-03, 4.36809127e-01,
  9.94229195e-01, 1.25195456e-01, 9.82290232e-01, 5.30693377e-01]
[2.41000000e+02,1.1759

[2.80000000e+02,6.64079660e-01,9.11561028e-01,6.30933716e-01,
 5.82564010e-01,1.39442491e-02,7.40823389e-01,7.00335223e-02,
 3.83514152e-02,8.28394288e-01,4.58552049e-01,6.08816632e-01]
[ 2.81000000e+02,-2.15196718e+00, 2.68730233e-01, 3.82297233e-01,
  5.57443891e-01, 2.03609964e-01, 4.43462095e-01, 6.55935579e-02,
  6.11400743e-01, 2.69208097e-01, 2.21432421e-01, 6.59856609e-02]
[ 2.82000000e+02,-5.68536653e-01, 3.70414548e-01, 7.38010509e-01,
  3.73694817e-01, 1.70245997e-01, 9.32435892e-01, 8.48747450e-01,
  5.80392555e-01, 4.49380930e-01, 3.49884420e-01, 2.98601847e-01]
[283.        ,  0.67554441,  0.92154136,  0.81822101,  0.33619429,
   0.83280547,  0.94662822,  0.75858895,  0.78772621,  0.61639626,
   0.82517789,  0.60204049]
[ 2.84000000e+02,-6.06648764e-02, 7.71874323e-01, 6.17060047e-01,
  3.47630931e-01, 9.95588601e-01, 7.61126092e-01, 3.55230319e-01,
  6.59631234e-02, 4.30435211e-01, 1.09569155e-01, 3.73620897e-01]
[ 2.85000000e+02,-1.26461829e+00, 8.16706402e-01, 1.249858

KeyboardInterrupt: 

**N.B** As the cell runs an infinite loop, the producer is never going to stop by itself. 
Don't forget to stop the cell using the dedicated button (■).